# 2. Trích xuất và xử lý dữ liệu mô hình

### Thiết lập ban đầu

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

pd.options.mode.copy_on_write = True

In [1]:
# # For Google Colab
# from google.colab import drive
# from pydrive2.auth import GoogleAuth
# from pydrive2.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# # Auth Google Drive
# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# # Download files on Shared folder to Google Colab

# # Shared folder ID
# folder_id = "1cv2YpR9aqRwocc8jwHLcFopfV9G0S-vH"

# # File name
# needed_files = [
#     "chotot_data.csv",
#     "input_scoli_2023.json",
#     "model_ref_price_extra.csv",
#     "model_ref_price_full.csv",
#     "origin_country_multiplier.csv",
# ]

# # Search for the file in the shared folder
# file_list = drive.ListFile({'q': f"'{folder_id}' in parents"}).GetList()
# file_list_simplified = []
# if file_list:
#     for file in file_list:
#       file_name = file['title']
#       file_id = file['id']
#       if file_name in needed_files:
#         file_list_simplified.append({
#             file_name: file_id
#         })

#         # Download the file to Colab
#         downloaded = drive.CreateFile({'id': file_id})
#         downloaded.GetContentFile(file_name)

# MODEL_REF_PRICE_PATH = "model_ref_price_full.csv"
# MODEL_REF_EXTRA_PRICE_PATH = "model_ref_price_extra.csv"
# COUNTRY_MULTIPLIER_PATH = "origin_country_multiplier.csv"
# SCOLI_PATH = "input_scoli_2023.json"
# INPUT_FILE_PATH = "chotot_data.csv"


# For VSCode
MODEL_REF_PRICE_PATH = "../data/raw/model_ref_price_full.csv"
MODEL_REF_EXTRA_PRICE_PATH = "../data/raw/model_ref_price_extra.csv"
COUNTRY_MULTIPLIER_PATH = "../data/raw/origin_country_multiplier.csv"
SCOLI_PATH = "../data/raw/input_scoli_2023.json"
INPUT_FILE_PATH = "../data/raw/chotot_data.csv"

## 2.2. Tiền xử lý dữ liệu

## 2.3. EDA các biến

# 3. Tạo mô hình

## 3.1. Các loại mô hình phù hợp (slide)

## 3.2. Mô hình hồi quy

## 3.3. Mô hình Random Forest

## 3.4. So sánh hai mô hình (slide)